<a href="https://colab.research.google.com/github/beir-nlp/beir/blob/main/Retrieval_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BEIR Retrieval Example** 

This notebook shows the usage of BEIR for passage retrieval.

Here, we show retrieval using various dense retriever models including Sentence-Transformers, USE-QA and DPR.

In [18]:
!nvidia-smi

Wed Jan 27 14:17:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    64W / 149W |   8538MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install beir

     |████████████████████████████████| 3.4MB 6.3MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 1.8MB 36.0MB/s 
     |████████████████████████████████| 1.2MB 37.4MB/s 
     |████████████████████████████████| 890kB 41.8MB/s 
     |████████████████████████████████| 2.9MB 39.5MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp36-cp36m-linux_x86_64.whl size=264228 sha256=90e3fa357531d19d06e2d124f0fae63d2a201568234848cf798a39851325d35b
  Stored in directory: /root/.cache/pip/wheels/55/66/40/1779aa0a8eb66e088669befe286f695cdfe420ba91ce662127
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=985f3dbbf080b598343a52cb3f7c5107eb48994f201defed296ce1435f300c1e
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a6a

# **Download Dataset**

You can download any dataset from the ones mentioned below, we use NFCorpus for our example.

Warning: Exact search on big datasets can often take time (few hours)!

We Provide IR evaluation for the following datasets:

*   TREC-COVID
*   NFCorpus
*   NQ
*   HotpotQA
*   NewsQA
*   FiQA
*   ArguAna
*   Touche-2020
*   CQaDupstack
*   Quora
*   DBPedia-v2
*   SCIDOCS
*   FEVER
*   Climate-FEVER



In [5]:
import pathlib, os

from beir import util

dataset = "nfcorpus.zip"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")

data_path = util.download_and_unzip(url, out_dir)

Unzipping nfcorpus.zip ...


# **Folder Structure**

* nfcorpus/
    * corpus.jsonl
    * queries.jsonl
    * qrels/

        * train.tsv
        * dev.tsv
        * test.tsv

In [19]:
!ls datasets/nfcorpus/

corpus.jsonl  qrels  queries.jsonl


# **Data Loading**

In [8]:
from beir.datasets.data_loader import GenericDataLoader

data_path = "datasets/nfcorpus"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")

# **Data Format**


# **Exact Search using Sentence-Transformers**

We use the [``distilroberta-base-msmarco-v2``](https://www.sbert.net/docs/pretrained-models/msmarco-v2.html) model in this example.

In [10]:
from beir.retrieval.evaluation import EvaluateRetrieval

retriever = EvaluateRetrieval(model="sbert", model_name="distilroberta-base-msmarco-v2")
results = retriever.retrieve(corpus, queries, qrels)

100%|██████████| 305M/305M [00:35<00:00, 8.66MB/s]


Encoding Queries...



Encoding Corpus...


# **Exact Search using DPR**

We use the [``facebook/dpr-question_encoder-single-nq-base``](https://huggingface.co/transformers/model_doc/dpr.html#dprquestionencoder) and the [``facebook/dpr-ctx_encoder-single-nq-base``](https://huggingface.co/transformers/model_doc/dpr.html#dprcontextencoder) models in this example.

In [21]:
import tqdm
from beir.retrieval.evaluation import EvaluateRetrieval

retriever = EvaluateRetrieval(model="dpr")
results = retriever.retrieve(corpus, queries, qrels)


que:   0%|          | 0/3 [00:00<?, ?it/s]

Encoding Queries...



que: 100%|██████████| 3/3 [00:00<00:00,  4.10it/s]

pas:   0%|          | 0/29 [00:00<?, ?it/s]

Encoding Corpus...



pas:  76%|███████▌  | 22/29 [02:55<00:58,  8.34s/it]

RuntimeError: ignored

# **Exact Search using USE-QA**

We use the [``universal-sentence-encoder-qa/3``](https://tfhub.dev/google/universal-sentence-encoder-qa/3) model in our example.

In [22]:
import tqdm
from beir.retrieval.evaluation import EvaluateRetrieval

retriever = EvaluateRetrieval(model="use-qa")
results = retriever.retrieve(corpus, queries, qrels)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-qa/3'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-qa/3: 550.04MB
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-qa/3, Total size: 588.94MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-qa/3'.


InternalError: ignored

# **Evaluation**

We evaluate IR models ``NDCG@k``, ``MAP@K`` and ``Recall@K``.

For this Example, we chose to evaluate at k = ``[1,3,5,10,100,1000]``

In [23]:
ndcg, _map, recall = retriever.evaluate(qrels, results, retriever.k_values)

In [24]:
print(ndcg)

{'ndcg@1': 0.33282, 'ndcg@3': 0.27662, 'ndcg@5': 0.25527, 'ndcg@10': 0.23186, 'ndcg@100': 0.21083, 'ndcg@1000': 0.30276}


In [25]:
print(_map)

{'map@1': 0.04549, 'map@3': 0.06389, 'map@5': 0.07227, 'map@10': 0.08216, 'map@100': 0.09821, 'map@1000': 0.1084}


In [26]:
print(recall)

{'recall@1': 0.04549, 'recall@3': 0.06965, 'recall@5': 0.08473, 'recall@10': 0.10574, 'recall@100': 0.21654, 'recall@1000': 0.53311}
